In [ ]:
from scipy.io import loadmat


filename = "D:/数据/Datasets/UoC齿轮故障数据集/DataForClassification_TimeDomain.mat"  # 要处理的文件路径
signal = loadmat(filename)
print(signal)

In [ ]:
import numpy as np

# signal = loadmat('../datasets/A/None/97.mat')
drive_end_key = [k for k in signal.keys() if 'AccTimeDomain' in k]
print(signal[drive_end_key[0]].shape)
de_signal = signal[drive_end_key[0]].ravel()
print(len(de_signal))

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
from scipy.signal import stft

filename = 'D:/数据/data/ok/ok-05-29-14-30-55.csv'
data = pd.read_csv(filename,index_col=0)
data['cDAQ9181-171B2ECMod1/ai0']

print(data['cDAQ9181-171B2ECMod1/ai0'].values[0])

In [ ]:
print(len(data['cDAQ9181-171B2ECMod1/ai0'].values))

In [ ]:
ir_fault_data = data['cDAQ9181-171B2ECMod1/ai0'].values
fs = 10240 # 12KHz
f_ir7,t_ir7,Zxx_ir7 = stft(ir_fault_data,fs,window='hann', nperseg=1024,noverlap=512)

f_mask = (f_ir7 >= 0) & (f_ir7 <= 500)
f_filtered = f_ir7[f_mask]
Zxx_filtered = Zxx_ir7[f_mask, :]

In [ ]:
import numpy as np


# 绘制频谱图
plt.pcolormesh(t_ir7, f_filtered, np.abs(Zxx_filtered), shading='gouraud')
plt.colorbar(label='Magnitude')
plt.title('STFT Magnitude Spectrum')
plt.xlabel('Time [s]')
plt.ylabel('Frequency [Hz]')
plt.show()

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.signal import stft
import matplotlib.pyplot as plt

# 指定目录路径
directory = 'C:/Users/black/Desktop/data/try/ng'  # 替换为实际的目录路径

# 获取目录下的所有文件
files = os.listdir(directory)

# 指定保存图片的文件夹路径
output_folder = 'D:/数据/data/测试数据/ng'  # 替换为实际的文件夹路径

# 循环遍历文件
for file in files:
    if file.endswith('.csv'):  # 仅处理CSV文件
        file_path = os.path.join(directory, file)  # 构建文件路径
        
        # 读取CSV文件,取消索引
        data = pd.read_csv(file_path,index_col=0) 
        
        # 使用stft变换
        ir_fault_data = data['cDAQ9181-171B2ECMod1/ai0'].values
        fs = 10240 # 12KHz
        f_ir7,t_ir7,Zxx_ir7 = stft(ir_fault_data,fs,window='hann', nperseg=1024,noverlap=512)

        # 筛选频率范围为0 Hz到1000 Hz的数据
        f_mask = (f_ir7 >= 0) & (f_ir7 <= 500)
        f_filtered = f_ir7[f_mask]
        Zxx_filtered = Zxx_ir7[f_mask, :]

        #保存转换图
        image_name = file[:-4] + '.png'
        output_file = os.path.join(output_folder, image_name)
        fig, ax = plt.subplots(figsize=(2.24, 2.24))  # 设置图形大小为224x224
        ax.pcolormesh(t_ir7, f_filtered, np.abs(Zxx_filtered), shading='gouraud')
        # 移除坐标轴
        ax.axis('off')
        # 调整图像边界
        fig.tight_layout(pad=0)
        # # 添加colorbar
        # cbar = fig.colorbar(pcm, ax=ax, label='Magnitude')
        # ax.set_title('STFT Magnitude Spectrum')
        # ax.set_xlabel('Time [s]')
        # ax.set_ylabel('Frequency [Hz]')
        file_name = file[:-4] + '.png'
        output_file = os.path.join(output_folder, file_name)
        plt.savefig(output_file, dpi=100)  # 设置图片分辨率为100 dpi
        plt.close(fig)

In [ ]:
import os
import random
import shutil

# 定义源图像文件夹路径和目标文件夹路径
source_dir = 'D:/数据/data/ng'  # 替换为实际的图像文件夹路径
train_dir = 'D:/数据/data/image/train'  # 替换为实际的训练集文件夹路径

# 创建train文件夹下的ng和ok子文件夹
train_ok_dir = os.path.join(train_dir, 'ng')
os.makedirs(train_ok_dir, exist_ok=True)

# 定义移动比例
move_ratio = 0.9

# 移动ok子文件夹下的图像
ok_dir = os.path.join(source_dir)
ok_images = os.listdir(ok_dir)
random.shuffle(ok_images)
ok_move_count = int(len(ok_images) * move_ratio)

for i in range(ok_move_count):
    image_name = ok_images[i]
    source_path = os.path.join(ok_dir, image_name)
    target_path = os.path.join(train_ok_dir, image_name)
    shutil.move(source_path, target_path)



In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# 定义数据集路径
data_dir = 'D:/数据/data/image'  # 替换为实际的图像文件夹路径

# 定义数据预处理
data_transform = transforms.Compose([
    # transforms.Resize((224, 224)),  # 将图像大小调整为224x224
    transforms.ToTensor(), # 将图像转换为Tensor
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 归一化操作
])

# 加载训练集和测试集
train_dataset = ImageFolder(data_dir + '/train', transform=data_transform)
test_dataset = ImageFolder(data_dir + '/test', transform=data_transform)

# 打印训练集类别个数
num_classes = len(train_dataset.classes)
print(f"训练集类别个数: {num_classes}")

# 定义数据加载器
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# 加载预训练的ResNet-34模型
model = models.resnet34(pretrained=True)

# 替换最后一层全连接层
model.fc = nn.Linear(model.fc.in_features, num_classes)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 训练模型
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = correct / total

    print(f"Epoch [{epoch+1}/{num_epochs}]: Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

# 在测试集上评估模型
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

test_accuracy = correct / total
print(f"Test Accuracy: {test_accuracy:.4f}")

训练集类别个数: 2


d:\Anaconda3\envs\load_data\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
d:\Anaconda3\envs\load_data\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10]: Train Loss: 0.4731, Train Accuracy: 0.7688
Epoch [2/10]: Train Loss: 0.2014, Train Accuracy: 0.9282
Epoch [3/10]: Train Loss: 0.0947, Train Accuracy: 0.9692
Epoch [4/10]: Train Loss: 0.0575, Train Accuracy: 0.9806
Epoch [5/10]: Train Loss: 0.0273, Train Accuracy: 0.9954
Epoch [6/10]: Train Loss: 0.0159, Train Accuracy: 0.9989
Epoch [7/10]: Train Loss: 0.0074, Train Accuracy: 1.0000
Epoch [8/10]: Train Loss: 0.0135, Train Accuracy: 0.9977
Epoch [9/10]: Train Loss: 0.0104, Train Accuracy: 1.0000
Epoch [10/10]: Train Loss: 0.0050, Train Accuracy: 1.0000
Test Accuracy: 0.9490


In [5]:
import os
from PIL import Image
# 使用模型对新图片进行预测

img_dir = 'D:/数据/data/测试数据/ng'  # 预测路径
count = 18
right = 0

# 获取目录下的所有文件
img_files = os.listdir(img_dir)
print(img_files)
# 循环遍历文件
for file in img_files:
    if file.endswith('.png'):  # 仅处理CSV文件
        file_path = os.path.join(img_dir, file)  # 构建文件路径
        new_image_path = file_path
        print(new_image_path)
        new_image = Image.open(new_image_path).convert('RGB')
        new_image_tensor = data_transform(new_image).unsqueeze(0).to(device)

        model.eval()
        with torch.no_grad():
            output = model(new_image_tensor)
        _, predicted_idx = torch.max(output, 1)
        predicted_label = train_dataset.classes[predicted_idx.item()]
        print("预测类别为:", predicted_label) 
        # print(file[file.find("ok"):file.find("ok") + 1])
        if predicted_label == "ng":
            right += 1
print("预测准确度：" ,(right / count))

['tryng-05-30-10-58-19.png', 'tryng-05-30-10-58-31.png', 'tryng-05-30-10-58-44.png', 'tryng-05-30-10-58-55.png', 'tryng-05-30-10-59-08.png', 'tryng-06-02-01-02-29.png', 'tryng-06-02-01-02-58.png', 'tryng-06-02-01-03-08.png', 'tryng-06-02-01-03-18.png', 'tryng-06-02-01-03-28.png', 'tryng-06-02-01-03-38.png', 'tryng-06-02-01-03-48.png', 'tryng-06-02-01-03-58.png', 'tryng-06-02-01-04-09.png', 'tryng-06-02-01-04-20.png', 'tryng-06-02-01-04-30.png', 'tryng-06-02-01-04-44.png', 'tryng-06-02-01-04-55.png']
D:/数据/data/测试数据/ng\tryng-05-30-10-58-19.png
预测类别为: ng
D:/数据/data/测试数据/ng\tryng-05-30-10-58-31.png
预测类别为: ng
D:/数据/data/测试数据/ng\tryng-05-30-10-58-44.png
预测类别为: ng
D:/数据/data/测试数据/ng\tryng-05-30-10-58-55.png
预测类别为: ng
D:/数据/data/测试数据/ng\tryng-05-30-10-59-08.png
预测类别为: ng
D:/数据/data/测试数据/ng\tryng-06-02-01-02-29.png
预测类别为: ng
D:/数据/data/测试数据/ng\tryng-06-02-01-02-58.png
预测类别为: ng
D:/数据/data/测试数据/ng\tryng-06-02-01-03-08.png
预测类别为: ng
D:/数据/data/测试数据/ng\tryng-06-02-01-03-18.png
预测类别为: ng
D:/数据/dat

In [6]:
import os


torch.save(model.state_dict(), 'D:/数据/data/model/model_improve.pth')

In [7]:
print(model)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  